Automobile Sales Forecasting Using Functional Data Analysis

In [ ]:
#合并后的完整数据集FullData.csv

In [4]:
import pandas as pd
FullData = pd.read_csv('FullData.csv')
del FullData['Unnamed: 0']
FullData.columns

Index(['TYPE', '厂商', '品牌', '国别', '城市', '城市级别', '子车型ID', '子车型', '子车型英文名', '年',
       '月', '燃料类型', '省份', '车身形式', '销量', '集团', '是否新能源', '细分市场', '生厂商品牌属性',
       'MSRP', 'TP', '车身类型', '级别', '厂商子品牌', '厂商子品牌E', '品牌挂标', '跨国公司', '变速箱形式',
       '行李厢容积(L)', '进气形式', '燃油标号', '供油方式', '环保标准', '挡位个数', '驱动方式', '助力类型',
       '电机类型', '驱动电机数', '电机布局', '电池类型', '电池组质保', '最大功率(kW)', '最大扭矩(N?m)',
       '最高车速(km/h)', '长度(mm)', '宽度(mm)', '高度(mm)', '轴距(mm)', '车门数(个)',
       '座位数(个)', '整备质量(kg)', '排量(L)', '气缸数(个)', '每缸气门数(个)', '最大马力(Ps)',
       '最大功率转速(rpm)', '电动机总功率', '电动机总扭矩', '工信部纯电续航里程', '电池容量', '快充时间', '慢充时间'],
      dtype='object')

In [5]:
FDAData = FullData.iloc[:,[6,9,10,11,12,14,19,20]]
FDAData['MonthNum'] = (FDAData['年']-2018)*12 + FDAData['月']
FDAData.head()

<ipython-input-5-039c338d2c59>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FDAData['MonthNum'] = (FDAData['年']-2018)*12 + FDAData['月']


,子车型ID,年,月,燃料类型,省份,销量,MSRP,TP,MonthNum
0,54,2018,1,汽油,广西,11.0,259500.0,251000.0,1
1,295,2018,1,汽油,山东,1.0,266200.0,250100.0,1
2,517,2018,1,汽油,安徽,55.0,1275600.0,1128400.0,1
3,322,2018,1,非插电式混合动力,贵州,9.0,268100.0,237300.0,1
4,329,2018,1,汽油,湖南,2.0,80300.0,59300.0,1


In [ ]:
FDAData = FDAData.dropna(axis=0,how='any',subset=['MSRP'])
FDAData.iloc[:,[0,3,4,5,8]].groupby(by=['子车型ID','燃料类型','省份','MonthNum']).sum()
Sales = pd.DataFrame(FDAData.iloc[:,[0,3,4,5,8]].groupby(by=['子车型ID','燃料类型','省份','MonthNum']).sum())
Sales.to_csv('Sales.csv',encoding = "utf_8_sig")
Sales = pd.read_csv('Sales.csv')
sum(Sales.iloc[:,[0,1,2]].value_counts()==40) #2276
cnt = Sales.iloc[:,[0,1,2]].value_counts()
cnt = cnt[0:2276]
cnt = pd.DataFrame(cnt)
cnt.to_csv('cnt.csv',encoding = "utf_8_sig")
cnt= pd.read_csv('cnt.csv')

In [ ]:
grid_points_train = []
for x in range(1,37):
    grid_points_train.append(x)

data_matrix_train = []
for i in cnt.index:
    ID = cnt['子车型ID'][i]
    Fuel = cnt['燃料类型'][i]
    Prov = cnt['省份'][i]
    
    ts = []
    for j in range(1,37):
        s = Sales.loc[Sales['子车型ID'] == ID].loc[Sales['燃料类型'] == Fuel].loc[Sales['省份'] == Prov].loc[Sales['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        ts.append(ss['销量'][0])
        
    data_matrix_train.append(ts)

data_matrix_train = pd.DataFrame(data_matrix_train)
data_matrix_train.to_csv('data_matrix_train.csv',encoding = "utf_8_sig")

In [ ]:
grid_points_test = []
for x in range(37,41):
    grid_points_test.append(x)

data_matrix_test = []
for i in cnt.index:
    ID = cnt['子车型ID'][i]
    Fuel = cnt['燃料类型'][i]
    Prov = cnt['省份'][i]
    
    ts = []
    for j in range(37,41):
        s = Sales.loc[Sales['子车型ID'] == ID].loc[Sales['燃料类型'] == Fuel].loc[Sales['省份'] == Prov].loc[Sales['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        ts.append(ss['销量'][0])
        
    data_matrix_test.append(ts)
    
data_matrix_test = pd.DataFrame(data_matrix_test)
data_matrix_test.to_csv('data_matrix_test.csv',encoding = "utf_8_sig")

In [ ]:
FDAData = FullData.dropna(axis=0,how='any',subset=['MSRP'])
MSRP = FDAData.iloc[:,[0,3,4,6,8]].groupby(by=['子车型ID','燃料类型','省份','MonthNum']).mean()
MSRP.to_csv('MSRP.csv',encoding = "utf_8_sig")
MSRP = pd.read_csv('MSRP.csv')

In [ ]:
data_matrix_train_MSRP = []
for i in cnt.index:
    ID = cnt['子车型ID'][i]
    Fuel = cnt['燃料类型'][i]
    Prov = cnt['省份'][i]
    
    ts = []
    for j in range(1,37):
        s = MSRP.loc[MSRP['子车型ID'] == ID].loc[MSRP['燃料类型'] == Fuel].loc[MSRP['省份'] == Prov].loc[MSRP['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        ts.append(ss['MSRP'][0])
        
    data_matrix_train_MSRP.append(ts)
    
data_matrix_train_MSRP = pd.DataFrame(data_matrix_train_MSRP)
data_matrix_train_MSRP.to_csv('data_matrix_train_MSRP.csv',encoding = "utf_8_sig")

In [ ]:
data_matrix_test_MSRP = []
for i in cnt.index:
    ID = cnt['子车型ID'][i]
    Fuel = cnt['燃料类型'][i]
    Prov = cnt['省份'][i]
    
    ts = []
    for j in range(37,41):
        s = MSRP.loc[MSRP['子车型ID'] == ID].loc[MSRP['燃料类型'] == Fuel].loc[MSRP['省份'] == Prov].loc[MSRP['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        ts.append(ss['MSRP'][0])
        
    data_matrix_test_MSRP.append(ts)

data_matrix_test_MSRP = pd.DataFrame(data_matrix_test_MSRP)
data_matrix_test_MSRP.to_csv('data_matrix_test_MSRP.csv',encoding = "utf_8_sig")

In [6]:
#city

In [ ]:
FDAData = FullData.iloc[:,[6,9,10,11,4,14,19,20]]
FDAData['MonthNum'] = (FDAData['年']-2018)*12 + FDAData['月']
Sales_city = pd.DataFrame(FDAData.iloc[:,[0,3,4,5,8]].groupby(by=['子车型ID','燃料类型','城市','MonthNum']).sum())
Sales_city.to_csv('Sales_city.csv',encoding = "utf_8_sig")
Sales_city = pd.read_csv('Sales_city.csv')
sum(Sales_city.iloc[:,[0,1,2]].value_counts()>=28) #5716
cnt_city = Sales_city.iloc[:,[0,1,2]].value_counts()
cnt_city = cnt_city[0:5716]
cnt_city = pd.DataFrame(cnt_city)
cnt_city.to_csv('cnt_city.csv',encoding = "utf_8_sig")
cnt_city = pd.read_csv('cnt_city.csv')

grid_points = []
for x in range(25,41):
    grid_points.append(x)
    
data_matrix_city = []
for i in cnt_city.index:
    ID = cnt_city['子车型ID'][i]
    Fuel = cnt_city['燃料类型'][i]
    City = cnt_city['城市'][i]
    
    ts = []
    for j in range(25,41):
        s = Sales_city.loc[Sales_city['子车型ID'] == ID].loc[Sales_city['燃料类型'] == Fuel].loc[Sales_city['城市'] == City].loc[Sales_city['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        if 0 in ss.index:
            ts.append(ss['销量'][0])
        else:
            ts.append(-1)
            break
    if -1 not in ts:
        data_matrix_city.append(ts)
        
data_matrix_city = pd.DataFrame(data_matrix_city)
data_matrix_city.to_csv('data_matrix_city.csv',encoding = "utf_8_sig")

In [7]:
#citylevel

In [ ]:
FDAData = FullData.iloc[:,[6,9,10,11,5,14,19,20]]
FDAData['MonthNum'] = (FDAData['年']-2018)*12 + FDAData['月']
citylevel = pd.DataFrame(FDAData.iloc[:,[0,3,4,5,8]].groupby(by=['子车型ID','燃料类型','城市级别','MonthNum']).sum())
citylevel.to_csv('citylevel.csv',encoding = "utf_8_sig")
citylevel = pd.read_csv('citylevel.csv')
sum(citylevel.iloc[:,[0,1,2]].value_counts() == 40) #997
cnt_citylevel = citylevel.iloc[:,[0,1,2]].value_counts()
cnt_citylevel = cnt_citylevel[0:997]
cnt_citylevel.to_csv('cnt_citylevel.csv',encoding = "utf_8_sig")
cnt_citylevel = pd.read_csv('cnt_citylevel.csv')

grid_points = []
for x in range(1,41):
    grid_points.append(x)

data_matrix_citylevel = []
for i in cnt_citylevel.index:
    ID = cnt_citylevel['子车型ID'][i]
    Fuel = cnt_citylevel['燃料类型'][i]
    Citylevel = cnt_citylevel['城市级别'][i]
    
    ts = []
    for j in range(1,41):
        s = citylevel.loc[citylevel['子车型ID'] == ID].loc[citylevel['燃料类型'] == Fuel].loc[citylevel['城市级别'] == Citylevel].loc[citylevel['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        if 0 in ss.index:
            ts.append(ss['销量'][0])
        else:
            ts.append(-1)
            break
    if -1 not in ts:
        data_matrix_citylevel.append(ts)
        
data_matrix_citylevel = pd.DataFrame(data_matrix_citylevel)
data_matrix_citylevel.to_csv('data_matrix_citylevel.csv',encoding = "utf_8_sig")

In [8]:
#submarket

In [ ]:
FDAData = FullData.iloc[:,[6,9,10,11,17,14,19,20]]
FDAData['MonthNum'] = (FDAData['年']-2018)*12 + FDAData['月']
submarket = pd.DataFrame(FDAData.iloc[:,[0,3,4,5,8]].groupby(by=['子车型ID','燃料类型','细分市场','MonthNum']).sum())
submarket.to_csv('submarket.csv',encoding = "utf_8_sig")
submarket = pd.read_csv('submarket.csv')
sum(submarket.iloc[:,[0,1,2]].value_counts() == 40) #257
cnt_submarket.to_csv('cnt_submarket.csv',encoding = "utf_8_sig")
cnt_submarket = pd.read_csv('cnt_submarket.csv')

grid_points = []
for x in range(1,41):
    grid_points.append(x)
    
data_matrix_submarket = []
for i in cnt_submarket.index:
    ID = cnt_submarket['子车型ID'][i]
    Fuel = cnt_submarket['燃料类型'][i]
    Submarket = cnt_submarket['细分市场'][i]
    
    ts = []
    for j in range(1,41):
        s = submarket.loc[submarket['子车型ID'] == ID].loc[submarket['燃料类型'] == Fuel].loc[submarket['细分市场'] == Submarket].loc[submarket['MonthNum'] == j]
        ss = s.reset_index(drop=True)
        if 0 in ss.index:
            ts.append(ss['销量'][0])
        else:
            ts.append(-1)
            break
    if -1 not in ts:
        data_matrix_submarket.append(ts)
        
data_matrix_submarket = pd.DataFrame(data_matrix_submarket)
data_matrix_submarket.to_csv('data_matrix_submarket.csv',encoding = "utf_8_sig")

In [9]:
#new

In [ ]:
len(FullData) #2621796
FullData = FullData.dropna(axis=0,how='any',subset=['MSRP']) #2042941
new = FullData[FullData['是否新能源']=='是'] #67444
new.to_csv('new.csv',encoding = "utf_8_sig")

In [10]:
#traditional

In [ ]:
traditional = FullData[FullData['是否新能源']=='否'] #1975497
del traditional['电机类型']
del traditional['驱动电机数']
del traditional['电机布局']
del traditional['电池类型']
del traditional['电池组质保']
del traditional['电动机总功率']
del traditional['电动机总扭矩']
del traditional['工信部纯电续航里程']
del traditional['电池容量']
del traditional['快充时间']
del traditional['慢充时间']
traditional.to_csv('traditional.csv',encoding = "utf_8_sig")